# Spark (load in the data)

In [2]:
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC") # fix timestamps loaded by spark
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/07/27 05:21:49 WARN Utils: Your hostname, JamesPanzera resolves to a loopback address: 127.0.1.1; using 172.28.10.204 instead (on interface eth0)
23/07/27 05:21:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/27 05:21:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/27 05:21:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
sdf_yellow_2022_01 = spark.read.parquet('../data/raw/yellow/2022/2022-01.parquet')

sdf_yellow_2022_all = spark.read.parquet('../data/raw/yellow/2022')

#sdf_green_all = spark.read.parquet('../data/raw/green')

#sdf_fhv_all = spark.read.parquet('../data/raw/fhv')

#sdf_fhvhv_all = spark.read.parquet('../data/raw/fhvhv')

# may not be feasible to combine the datasets for different vehicle types, double check data dictionaries first

#sdf_all = spark.read.parquet('../data/raw/')

In [5]:
sdf_yellow_2022_01.limit(5)

#sdf_green_all.limit(5)

#sdf_fhv_all.limit(5)

#sdf_fhvhv_all.limit(5)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.8,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.1,1.0,N,236,42,1,8.0,0.5,0.5,4.0,0.0,0.3,13.3,0.0,0.0
2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.0,0.0,0.3,11.8,2.5,0.0
2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.3,1.0,N,68,163,1,23.5,0.5,0.5,3.0,0.0,0.3,30.3,2.5,0.0


In [9]:
# check dataset count (shape)
#sdf_yellow_2022_01.count(), sdf_yellow_2022_all.count()
len(sdf_yellow_2022_01.columns)

19

In [ ]:
# show first 10 records / rows for passenger count and trip distance attributes
sdf_yellow_2022_01.select('passenger_count', 'trip_distance').limit(10) 

# access passenger count
sdf_yellow_2022_01['passenger_count']




In [ ]:
from pyspark.sql import functions as F

# F.col('passenger_count') # column type for use with sdf.filter(F.col('passenger_count') == 5).limit(5)) for example

# this example below retrieves records where passenger count and trip distance are non-zero
''' sdf.where(
    # != 0 is fine, but let's also take into account some more realistic filters
    (F.col('passenger_count') > 0) 
    & (F.col('trip_distance') > 0)
) '''

# sdf.groupby('passenger_count').mean('trip_distance').limit(5) # groups by passenger count and average trip distance for each

# group by passenger count and average trip amount for each passenger count and show max trip distance for each passenger count
# use alias to give a different name to the attribute columns in the aggregated results
''' aggregated_results = sdf \
                    .groupBy("passenger_count") \
                    .agg(
                        F.mean("total_amount").alias("avg_trip_amount_usd"),
                        F.max("trip_distance").alias("max_trip_distance_miles")
                    ) \
                    .orderBy("passenger_count")

aggregated_results.show()

'''

# save dataframe as parquet
# aggregated_results.write.mode('overwrite').parquet('../../data/tute_data/aggregated_results') 

# read parquet back into dataframe
# temp_results = spark.read.parquet('../../data/tute_data/aggregated_results')

To sample your data and convert it into a `pandas` dataframe, you can use the `.toPandas()` and save a sample of the `sdf` to read it in. We will also fix the random seed to be `0` just for consistency.

In [ ]:
SAMPLE_SIZE = 0.05

df = sdf.sample(SAMPLE_SIZE, seed=0).toPandas()

df.to_parquet('../../data/tute_data/sample_data.parquet')

## Filter / transform enriched data using business logic 

In [ ]:
# Remove certain features / records based on business logic / data limitations and remove certain values (e.g. NULL) 

# Discretise numeric features or convert categorical / ordinal features to numeric features depending on what model will be utilised, may also
# need to standardise / scale numeric features

We recommend you save every dataframe or aggregation as `parquet` so you don't keep running your notebook from top to bottom waiting 20 years for a result, or have so many variables and dataframes defined that you run out of memory for small transformations.

We strongly suggest you have a `code` folder in your Project 1 directory with the following structure:
- `preprocessing_notebook_part_1.ipynb`: outputs a structured parquet format and saves it.
- `preprocessing_notebook_part_2.ipynb`: reads in the output from above and does some aggregations and sampling before saving it.
- `data_analysis_xyz.ipynb`: conducts analysis on a single sample or aggregation from the output above.
- `data_analysis_abc.ipynb`: conducts analysis on another single sample or aggregation from the output above.
- `...`

This is a very basic version of what you call a "data pipeline" (or ETL pipeline, etc).

#### Read business rules and particular limitations of the data carefully (e.g. tip amount not recorded for cash payment) to guide self in determining which attributes may need to be filtered / removed or transformed / engineered